In [1]:
import pandas as pd
import jieba
import numpy as np

In [11]:
#加载数据
news = pd.read_csv("./sqlResult.csv",encoding='gb18030')

In [12]:
#清洗数据
news = news.dropna(subset=["content"])
#加载听哟用词

with open ("chinese_stopwords.txt",'r',encoding='utf-8')as file:
    stopwords = [ i[:-1] for i in file.readlines()]


In [4]:
#分词函数
def split_text(text):
    text = text.replace(' ','').replace('\n','')
    text2 = jieba.cut(text)
    result = ' '.join([w for w in text2 if w not in stopwords])
    return result

In [13]:
corups=list(map(split_text,[str(i) for i in news.content]))


In [14]:
#计算tfidf

from  sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer

counvectorizer =   CountVectorizer(encoding='gb18030',min_df=0.015)
tfidftransformer = TfidfTransformer()
countvector = counvectorizer.fit_transform(corups)
tfidf = tfidftransformer.fit_transform(countvector)

In [15]:
#标记是否新华社新闻


label = list(map(lambda source:1 if "新华社" in str(source) else 0 ,news.source ))


In [19]:
from  sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

# X_train,X_test,y_train,y_test = train_test_split(tfidf.toarray(),label,test_size=0.25)
X_train,X_test,y_train,y_test = train_test_split(tfidf.toarray(),label,test_size = 0.3)
model = MultinomialNB()
# model.fix(X_train,y_train)
model.fit(X_train,y_train)


MultinomialNB()

In [22]:
from  sklearn.metrics import accuracy_score,precision_score,recall_score

y_predict = model.predict(X_test)

print("准确率",accuracy_score(y_test,y_predict))
print("召回率",precision_score(y_test,y_predict))
print("准确度",precision_score(y_test,y_predict))

准确率 0.8861660987096527
召回率 0.9610865388911227
准确度 0.9610865388911227


In [26]:
#使用模型预测
prediction = model.predict(tfidf.toarray())
labels = np.array(label)
# compare_news_index = pd.DataFrame({{"prediction":prediction,"labels":labels}})
compare_news_index = pd.DataFrame({"prediction":prediction,"labels":labels})

In [27]:
compare_news_index[:10]

,prediction,labels
0,0,0
1,0,0
2,0,0
3,1,1
4,1,0
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0


In [28]:
copy_news_index = compare_news_index[(compare_news_index["prediction"]==1)&(compare_news_index["labels"]==0)].index

In [29]:
from sklearn.preprocessing import Normalizer
from sklearn.cluster import  KMeans

normalizer = Normalizer()
scaled_array = normalizer.fit_transform(tfidf.toarray())

#使用Kmeans进行分类

kemans = KMeans(n_clusters=25)
k_labels = kemans.fit_predict(scaled_array)

print(k_labels.shape)


(87054,)


In [30]:
id_class = {index:class_ for index,class_ in enumerate(k_labels)}

In [32]:
from collections import  defaultdict
#实际为新华社的新闻
xinhuashe_new_index = compare_news_index[(compare_news_index['labels']==1)].index
class_id = defaultdict(set)

for index,class_ in id_class.items():
    if index in xinhuashe_new_index.tolist():
        class_id[class_].add(index)

In [33]:
#查找相似文章
from sklearn.metrics.pairwise import  cosine_similarity

def find_similar_text(cpindex,top=10):
    dist_dict = {i:cosine_similarity(tfidf[cpindex],tfidf[i]) for i in class_id[id_class[cpindex]] }
    
    #从大到小排序
    
    return sorted(dist_dict.items(),key = lambda x:x[1][0],reverse=True)[:top]
    
    